# Neural Network with Cubic Data
- y = x^3 -3x^2 -9x -1
- 4 hidden layers

## 1.Import Required Libraries

In [2]:
!pip3 install visdom

     |████████████████████████████████| 676 kB 5.3 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=bae2693abec5514e82692c388a86bf430fc6a1a2189e0aac74a734c61c4052b6
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=8fddd6753319bdedac7b5dd89b814ffe5396af3b44fefc429ffac8f4c8e5d6da
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from visdom import Visdom

In [4]:
# 코렙에서 visdom 사용 방법

# !npm install -g npm to update!
! npm install -g localtunnel
get_ipython().system_raw('/usr/local/bin/python -m pip install visdom')
get_ipython().system_raw('/usr/local/bin/python -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

import time
time.sleep(5)
! cat url.txt

import visdom
time.sleep(5)
vis = visdom.Visdom(port='6006')
print(vis)
time.sleep(3)
vis.text('testing')
! cat visdomlog.txt
# 아래 출력 메세지의 Your url is 부분의 url 클릭함
# 나타난 웹 페이지의 'click to countinue' 버튼 클릭함

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.77s
your url is: https://mighty-mouse-100.loca.lt


Setting up a new session...


/usr/local/lib/python3.7/dist-packages/visdom/server.py:39: DeprecationWarning: zmq.eventloop.ioloop is deprecated in pyzmq 17. pyzmq now works with default tornado and asyncio eventloops.
  ioloop.install()  # Needs to happen before any tornado imports!
INFO:root:Application Started
INFO:tornado.access:200 POST /env/main (127.0.0.1) 0.81ms
INFO:tornado.access:101 GET /vis_socket (127.0.0.1) 0.48ms
INFO:root:Opened visdom socket from ip: 127.0.0.1
INFO:tornado.access:200 POST /events (127.0.0.1) 0.81ms


## 2. Data Generation

In [5]:
num_data = 1000
num_epoch = 5000

noise = init.normal(torch.FloatTensor(num_data, 1), std=0.5)

x = init.uniform(torch.Tensor(num_data, 1), -10, 10)
y = (x**3) - 3*(x**2) - 9*x - 1
y_noise = y + noise

x.size(), y.size()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  


(torch.Size([1000, 1]), torch.Size([1000, 1]))

In [6]:
# visualize data with visdom

input_data = torch.cat([x, y_noise], 1)

win = vis.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-10,
        xtickmax=10,
        xtickstep=1,
        ytickmin=0,
        ytickmax=500,
        ytickstep=1,
        markersymbol='dot',
        markersize=5,
        markercolor=np.random.randint(0, 255, num_data),
    ),
)

# Tensor(n,1) != Tensor(n, 1) 버그 있음
vis.line(
    X=x.reshape(x.shape[0]), 
    Y=y.reshape(y.shape[0]), 
    win=win)

'window_3a50f29cb43808'

## 3. Model & Optimizer

In [7]:
# fully connected model with 4 hidden layer

model = nn.Sequential(
        nn.Linear(1,20),
        nn.ReLU(),
        nn.Linear(20,10),
        nn.ReLU(),
        nn.Linear(10,5),
        nn.ReLU(),
        nn.Linear(5,1),
    ).cuda()

loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(),lr=0.0005)

## 4. Train 

In [8]:
# train
loss_arr = []
label = Variable(y_noise.cuda())

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(x.cuda()))

    loss = loss_func(output, label)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(loss)

    loss_arr.append(loss.cpu().data.numpy())

tensor(221.3009, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(221.2433, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(221.2014, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(221.1441, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(221.0830, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(221.0103, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(220.9137, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(220.7801, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(220.5865, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(220.2860, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(219.7807, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(218.8346, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(216.7748, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(211.2426, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(190.1620, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(142.1187, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(138.2787, device=

In [9]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[ 0.3069],
        [-1.1173],
        [ 1.0445],
        [-0.4155],
        [-0.7434],
        [ 0.5420],
        [ 0.5391],
        [-0.2918],
        [-0.2720],
        [ 0.8688],
        [-0.1829],
        [-0.7248],
        [-0.7490],
        [-0.5135],
        [-0.0887],
        [-0.2603],
        [ 1.4283],
        [-0.2405],
        [-0.3280],
        [ 0.0222]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-1.6434, -2.0219,  0.0897, -2.4349, -3.3300, -1.9867, -3.1909,  1.2718,
         1.2256, -0.7694, -0.8129, -0.7319, -2.6295, -3.0023,  0.3295,  1.2361,
         1.5515,  1.1564, -1.4570,  1.2889], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([[ 4.4290e-02,  1.4960e-02,  5.9021e-02, -2.0116e-01,  1.9249e-01,
         -1.3661e-01, -2.0147e-01, -1.5884e-01, -1.4107e-01, -2.0611e-01,
          7.3283e-02, -1.1561e-01, -2.5347e-03,  1.3111e-01, -8.5817e-03,
         -6.3180e-02, -4.0494e-02,  4.8494

## 5. Visualize Trained Output

In [10]:
win_2 = vis.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-10,
        xtickmax=10,
        xtickstep=1,
        ytickmin=0,
        ytickmax=500,
        ytickstep=1,
        markersymbol='dot',
        markercolor=np.random.randint(0, 255, num_data),
        markersize=5,
    ),
)

vis.line(
    X=x.reshape(x.shape[0]),     
    Y = output.cpu().data.reshape(output.cpu().data.shape[0]),
    win = win_2,    
)

'window_3a50f362ff00ea'

## 6. Visualize Loss Graph

In [11]:
x = np.reshape([i for i in range(num_epoch)],newshape=[num_epoch,1])
loss_data = np.reshape(loss_arr,newshape=[num_epoch,1])

win3 = vis.line(
    X = x.reshape(x.shape[0]),
    Y = loss_data.reshape(loss_data.shape[0]),
)